In [1]:
from langchain_community.document_loaders import PyPDFLoader,DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

f:\Medical_Bot\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'f:\\Medical_Bot'

In [4]:
#extracting text from pdf files
def load_pdf_files(data_path):
    Loader=DirectoryLoader(
        data_path,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents=Loader.load()
    return documents

In [5]:
extracted_data=load_pdf_files("data")

In [6]:
for i in extracted_data:
    print(i.metadata.get("source"))

data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medic

In [7]:
len(extracted_data)

637

In [8]:
from typing import List
from langchain_core.documents import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    
    """Given a list of document objects,return a new lisy of Documents 
    objects containing only source in metadata and the original page_content
    """
    
    minimal_docs: List[Document]= []
    for doc in docs:
        src= doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs

In [9]:
minimal_docs=filter_to_minimal_docs(extracted_data)

In [10]:
#now splitting the docs into smLLer chunks
def text_split(minimal_docs):
    text_splitter=RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts_chunk=text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [11]:
texts_chunk=text_split(minimal_docs)
print(len(texts_chunk))

5859


In [12]:
from langchain_community.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    #Download and return the Huggingface embedding model
    model_name= "sentence-transformers/all-MiniLM-L6-v2"
    embeddings=HuggingFaceEmbeddings(model_name=model_name)
    return embeddings

embedding=download_embeddings()

C:\Users\Utkarsh Kumar\AppData\Local\Temp\ipykernel_56948\2788972494.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name=model_name)


In [13]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [14]:
vector=embedding.embed_query("Hello World")

In [15]:
len(vector)

384

In [16]:
from dotenv import load_dotenv
import os
load_dotenv()


True

In [41]:
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
GEMINI_API_KEY=os.getenv("GEMINI_API_KEY")

os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY
os.environ["GEMINI_API_KEY"]=GEMINI_API_KEY


In [18]:
from pinecone import Pinecone
pinecone_api_key=PINECONE_API_KEY
pc=Pinecone(api_key=pinecone_api_key)

In [19]:
pc

In [20]:
from pinecone import ServerlessSpec
index_name="medical-bot"

# Check if index exists using list_indexes()
existing_indexes = [idx['name'] for idx in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws",region="us-east-1")
        
    )
    
index=pc.Index(index_name)

In [21]:
from langchain_pinecone import PineconeVectorStore

docsearch=PineconeVectorStore.from_existing_index(
    embedding=embedding,
    index_name=index_name
)

In [22]:
retriever=docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [23]:
retreived_docs=retriever.invoke("What is Acne?")

In [24]:
retreived_docs

[Document(id='8615638a-aa08-48c7-8bf6-f71869c5ee19', metadata={'source': 'data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='1dfbb189-0890-4590-9c05-58750d720737', metadata={'source': 'data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='4a97fe21-e08d-4c24-827c-6cb804a44eee', metadata={'source': 'data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25')]

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

chatModel=ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=GEMINI_API_KEY,
    temperature=0.3
)

In [ ]:
question="What is Acromegaly and gigantism?"
docs=retriever.get_relevant_documents(question)
response=chatModel.invoke(question)
print(response.content)

Acromegaly is a disorder where an abnormal chemical release from the pituitary gland in the brain leads to increased growth in bone and soft tissue, along with other bodily disturbances. The provided text defines Acromegaly but does not offer a separate definition for gigantism.
